In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import math
from tensorflow.keras import layers
import cv2
from skimage import exposure
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

In [ ]:
def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        images.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        labels.append((label))
        
    return images, labels

In [ ]:
train = []
test = []
label_train = []
label_test = []

In [ ]:
images, labels = load_images_from_path('dataset/train/A', 0)
    
train += images
label_train += labels

images, labels = load_images_from_path('dataset/train/N', 1)
    
train += images
label_train += labels

images, labels = load_images_from_path('dataset/train/O', 2)
    
train += images
label_train += labels

In [ ]:
images, labels = load_images_from_path('dataset/test/A', 0)
    
test += images
label_test += labels

images, labels = load_images_from_path('dataset/test/N', 1)
    
test += images
label_test += labels

images, labels = load_images_from_path('dataset/test/O', 2)
    
test += images
label_test += labels



In [ ]:
from tensorflow.keras.utils import to_categorical

train_norm = np.array(train) / 255
test_norm = np.array(test) / 255


label_train_encoded = to_categorical(label_train)
label_test_encoded = to_categorical(label_test)

In [ ]:
num_classes = 3
input_shape = (224, 224, 3)

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 100
image_size = 224

In [ ]:
class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super().__init__(**kwargs)

        # This is our tokenizer.
        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # After passing the images through our mini-network the spatial dimensions
        # are flattened to form sequences.
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # Positional embeddings are optional in CCT. Here, we calculate
        # the number of sequences and initialize an `Embedding` layer to
        # compute the positional embeddings later.
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None

In [ ]:

class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (tf.shape(x).shape[0] - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Rescaling(scale=1.0 / 255),
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)

In [ ]:
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
):
    inputs = layers.Input(input_shape)

    # Augment data.
    augmented = data_augmentation(inputs)

    # Encode patches.
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(augmented)

    # Apply positional embedding.
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings

    # Calculate Stochastic Depth probabilities.
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Create multiple layers of the Transformer block.
    for i in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Apply sequence pooling.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Classify outputs.
    logits = layers.Dense(num_classes)(weighted_representation)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
class_names = ["A", "N", "O"]

In [ ]:
def run_experiment(model, num_folds=5, patience=16):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold = 0
    histories = []
    all_true_labels = []
    all_pred_labels = []

    for train_index, val_index in kf.split(train_norm):

        fold += 1
        print(f"Training fold {fold}/{num_folds}...")

        x_train_fold, x_val_fold = train_norm[train_index], train_norm[val_index]
        y_train_fold, y_val_fold = label_train_encoded[train_index], label_train_encoded[val_index]

        optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

        model.compile(
            optimizer=optimizer,
            loss=keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[
                keras.metrics.CategoricalAccuracy(name="accuracy"),
                keras.metrics.TopKCategoricalAccuracy(k=5, name="top-5-accuracy"),
            ],
        )

        early_stopping = EarlyStopping(
            monitor='val_accuracy',
            patience=patience,
            restore_best_weights=True
        )

        history = model.fit(
            x=x_train_fold,
            y=y_train_fold,
            batch_size=batch_size,
            epochs=num_epochs,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

        histories.append(history)

        # Calculate evaluation metrics for this fold
        y_pred_fold = model.predict(test_norm)
        y_pred_labels_fold = np.argmax(y_pred_fold, axis=1)
        y_true_labels_fold = np.argmax(label_test_encoded, axis=1)

        accuracy = accuracy_score(y_true_labels_fold, y_pred_labels_fold)
        precision = precision_score(y_true_labels_fold, y_pred_labels_fold, average='weighted')
        recall = recall_score(y_true_labels_fold, y_pred_labels_fold, average='weighted')
        f1 = f1_score(y_true_labels_fold, y_pred_labels_fold, average='weighted')
        confusion = confusion_matrix(y_true_labels_fold, y_pred_labels_fold)

        print(f"Metrics for fold {fold}:")
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 Score:", f1)
        print("Confusion Matrix:")
        print(confusion)

        # Append true and predicted labels for this fold
        all_true_labels.extend(y_true_labels_fold)
        all_pred_labels.extend(y_pred_labels_fold)

        # Generate and print a classification report for each class
        class_report = classification_report(y_true_labels_fold, y_pred_labels_fold, target_names=class_names)
        print(f"Classification Report for Fold {fold}:")
        print(class_report)

    # Calculate overall evaluation metrics after cross-validation
    overall_accuracy = accuracy_score(all_true_labels, all_pred_labels)
    overall_precision = precision_score(all_true_labels, all_pred_labels, average='weighted')
    overall_recall = recall_score(all_true_labels, all_pred_labels, average='weighted')
    overall_f1 = f1_score(all_true_labels, all_pred_labels, average='weighted')
    overall_confusion = confusion_matrix(all_true_labels, all_pred_labels)

    print("Overall Metrics After Cross-Validation:")
    print("Overall Accuracy:", overall_accuracy)
    print("Overall Precision:", overall_precision)
    print("Overall Recall:", overall_recall)
    print("Overall F1 Score:", overall_f1)
    print("Overall Confusion Matrix:")
    print(overall_confusion)

    # Generate and print a classification report for each class after all folds
    overall_class_report = classification_report(all_true_labels, all_pred_labels, target_names=class_names)
    print("Overall Classification Report:")
    print(overall_class_report)

    return histories



In [ ]:

cct_model = create_cct_model()
histories = run_experiment(cct_model)